In [211]:
import pandas as pd
import re
import requests
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [212]:
URLS = [
    'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita', #DONE
    'https://en.wikipedia.org/wiki/List_of_countries_by_Internet_connection_speeds', #DONE
    'https://en.wikipedia.org/wiki/List_of_countries_by_alcohol_consumption_per_capita', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_military_expenditures', # DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index', # DONE
'https://en.wikipedia.org/wiki/Democracy_Index', # DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_tertiary_education_attainment', # DONE
'https://en.wikipedia.org/wiki/Importance_of_religion_by_country', # DONE
'https://en.wikipedia.org/wiki/Christianity_by_country', #DONE
'https://en.wikipedia.org/wiki/Islam_by_country', #DONE
'https://en.wikipedia.org/wiki/Buddhism_by_country', #DONE
# 'https://en.wikipedia.org/wiki/Jewish_population_by_country', ##########NOT DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_infant_and_under-five_mortality_rates', #DONE
'https://en.wikipedia.org/wiki/Age_of_criminal_responsibility', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_external_debt', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_income_equality', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_total_health_expenditure_per_capita', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate', #DONE
    'https://en.wikipedia.org/wiki/List_of_sovereign_states_and_dependencies_by_total_fertility_rate', #DONE
'https://en.wikipedia.org/wiki/Tobacco_consumption_by_country', #DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_obesity_rate', #DONE
# 'https://en.wikipedia.org/wiki/List_of_countries_by_number_of_Internet_users', #DONE
# 'https://en.wikipedia.org/wiki/List_of_countries_by_median_age', #DONE 25
'https://en.wikipedia.org/wiki/List_of_countries_by_economic_freedom',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_oil_production',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_population_growth_rate',#DONE 28
'https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_meat_consumption',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_incarceration_rate',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_literacy_rate',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_age_at_first_marriage',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_spending_on_education_(%25_of_GDP)',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_homeless_population',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_milk_consumption_per_capita',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_number_of_scientific_and_technical_journal_articles',#DONE
'https://en.wikipedia.org/wiki/Books_published_per_country_per_year',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_food_energy_intake',#DONE
'https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature'#DONE
]

In [213]:
INDEXES = [
    {'column_index': (['Country/Territory', 'United Nations'],['Country/Territory','Estimate']), 'tables_index': 0}, #1
    {'column_index': ['Country/Territory', 'Averagedownloadspeed(Mbit/s)(Ookla)'], 'tables_index': 0}, #2
    {'column_index': ['Country', '2016'], 'tables_index': 0}, #3
    {'column_index': ['Location', 'Rate'], 'tables_index': 1}, #4
    {'column_index': ['Country', '% of GDP'], 'tables_index': 0}, #5
    {'column_index': (['Nation', 'HDI'], ['Nation', '2021 data (2022 report)​']), 'tables_index': 1}, #6 We dont have 2019 data
    {'column_index': ['Country', '2020'], 'tables_index': 3}, #7
    {'column_index': (['Country', 'Age 25–34'], ['Country', 'Equivalent to a 2-year degree or higher (%)']), 'tables_index': 1}, #8
    {'column_index': ['Country/District', 'Yes, important'], 'tables_index': 0}, #9
    {'column_index': ['Country or entity', '% Christian'], 'tables_index': 2}, #10
    {'column_index': ['Country/Region', 'Muslim percentage of total population'], 'tables_index': 0}, #11
    {'column_index': (['Country/Territory', 'Most recent national census data'], ['Country/Territory', '% Buddhist']), 'tables_index': 0}, #12
#     {'column_index': (['Countries', 'Core population'], ['Countries', 'pct']), 'tables_index': 0}, #13 NOT DONE
    {'column_index': ['Country or territory', '2019 mortality rate, under-5 (per 1000 live births)'], 'tables_index': 0}, #14
    {'column_index': ['Country', 'Age (reduced)'], 'tables_index': 0}, #15
    {'column_index': (['Country', 'Annual'],['Country','Nominal (US$)']), 'tables_index': 0}, #16
    {'column_index': ['Country/Region', '% of GDP'], 'tables_index': 0}, #17
    {'column_index': (['Country', 'Gini'],['Country','%']), 'tables_index': 0}, #18
    {'column_index': ['Location', '2018'], 'tables_index': 1}, #19
    {'column_index': ['Country', 'All'], 'tables_index': 0}, #20
    {'column_index': ['Country', 'Fertility rate in 2020(births/woman)'], 'tables_index': 1}, #21
    {'column_index': ['Country', 'Cigarettes'], 'tables_index': 0}, #22
    {'column_index': ['Country', 'Obesity rate (%)'], 'tables_index': 0}, #23
#     {'column_index': ['Country or area', 'Pct'], 'tables_index': 2}, #24
#     {'column_index': (['Country/Territory', 'Median ages in years'],['Country/Territory','2020 medians'],['Country/Territory','Combined']), 'tables_index': 0}, #25
    {'column_index': ['Country', 'Score'], 'tables_index': 0}, #26
    {'column_index': ['Country', 'Oil production per capita2021 (bbl/day/million people)'], 'tables_index': 0}, #27
    {'column_index': (['Country (or territory)', 'UN2015–20'],['Unnamed: 0_level_1', 'Unnamed: 6_level_1']), 'tables_index': 0}, #28 unnamed 2nd level
    {'column_index': (['Countries', 'Life expectancy at birth'],['Countries','All']), 'tables_index': 2}, #29
    {'column_index': ['Country', 'kg/person (2002)'], 'tables_index': 0}, #30
    {'column_index': ['Location', 'Rate per 100,000'], 'tables_index': 0}, #31
    {'column_index': (['Country', 'Elderly(65+)'],['Country','Rate']), 'tables_index': 0}, #32
    {'column_index': ['Country', 'Women'], 'tables_index': None}, #33
    {'column_index': ['Country or subnational area', 'Expenditure on education(% of GDP)'], 'tables_index': 0}, #34
    {'column_index': ['Country', 'Homelessper 10k'], 'tables_index': 0}, #35
    {'column_index': ['Country', 'Milk consumption 2013 (kg/capita/yr)'], 'tables_index': 0}, #36
    {'column_index': ['Country', 'Scientific publications per capita (in ppm)'], 'tables_index': 0}, #37
    {'column_index': ['Country', 'Titles'], 'tables_index': 0}, #38
    {'column_index': (['Country', 'Average daily dietary energy consumption per capita'],['Country','kilocalories']), 'tables_index': 0}, #39
    {'column_index': ['Country', 'Average yearly temperature (1961–1990 Celsius)'], 'tables_index': 0}, #40
]


In [214]:
dataframes = {}

metadata = {}

def mid(x):
    if isinstance(x,str) and '-' in x:
        x_list = x.split('-')
        a = x_list[0]
        b = x_list[1]
        if a == '' and b != '':
            return x
        elif a== '' or b == '':
            return -1.0 * float('0.0'+a+b)
        else:
            return (float(a)+float(b))/2.0
    return x
    
# i = 28 -1
# for url in URLS:
for i,url in enumerate(URLS):
    tables_index = INDEXES[i]['tables_index']
    column_index = INDEXES[i]['column_index']
    actual_url = url
    page = requests.get(actual_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tables = soup.find_all('table', class_='wikitable')
    main_title = actual_url.split('/')[-1]
    temp_df = pd.DataFrame()
    if tables_index is None:
        for table in tables:
            data_frame_list = pd.read_html(str(table))
            temp_df2 = pd.DataFrame(data_frame_list[0])
            temp_df = pd.concat([temp_df,temp_df2], axis=0, ignore_index=True)
    else:
        table = tables[tables_index]
        data_frame_list = pd.read_html(str(table))
        temp_df = pd.DataFrame(data_frame_list[0])

    # remove references from titles
    temp_df.rename(columns=lambda x: re.sub('(\s*\[.+\]\s*)', '', x.strip() ), inplace=True)
    temp_df = temp_df.loc[:, column_index]
    
    # clean the table while we are at it...
    for i, y in enumerate(temp_df.columns):
        if i > 0:
            temp_df[y] = temp_df[y].replace('n.a.', '0.0', regex=True)
            temp_df[y] = temp_df[y].replace('(−|–)','-',regex=True)
            temp_df[y] = temp_df[y].replace('(\s|%|—|<|\+|,|\[.+\]|\(.+\)|.*[a-zA-Z/]+.*)','',regex=True)
            temp_df[y] = temp_df[y].replace('^\s*$', np.nan, regex=True)
        else:
            # index col
            temp_df[y] = temp_df[y].replace('( \*|\s\*|\[.+\]|\s?\(.+\))', '',regex=True) # DONT DELETE SPACE
    
    dataframes[main_title] = temp_df
#     print(temp_df)
    

joined_df = pd.DataFrame()
for i,key in enumerate(dataframes):
    value = dataframes[key]
    if i == 0:
        joined_df = value
        joined_df.set_index(joined_df.columns[0], inplace=True)
    else:
        col_name = (value.columns[0][0], value.columns[0][1]) if type(value.columns[0]) is tuple else value.columns[0]
        joined_df = joined_df.join(value.set_index(col_name), how='outer', rsuffix='_new')


for i,col in enumerate(joined_df.columns):
    joined_df = joined_df.applymap(lambda a: mid(a))




In [215]:
joined_df

,"(United Nations, Estimate)",Averagedownloadspeed(Mbit/s)(Ookla),2016,Rate,% of GDP,"(HDI, 2021 data (2022 report)​)",2020,"(Age 25–34, Equivalent to a 2-year degree or higher (%))","Yes, important",% Christian,...,"Rate per 100,000","(Elderly(65+), Rate)",Women,Expenditure on education(% of GDP),Homelessper 10k,Milk consumption 2013 (kg/capita/yr),Scientific publications per capita (in ppm),Titles,"(Average daily dietary energy consumption per capita, kilocalories)",Average yearly temperature (1961–1990 Celsius)
Afghanistan,508,NaN,0.2,6.7,NaN,0.478,2.85,NaN,97,0.02,...,87,20.3,15.5,4.1,1180.0,62.23,9.0,2795,2040.0,12.60
Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,5181,45.25,7.5,2.1,NaN,0.796,6.08,NaN,50,17.0,...,164,86.9,24.0,4.0,NaN,303.72,216.0,390,3360.0,11.40
Algeria,3368,NaN,0.9,1.3,NaN,0.745,3.77,NaN,95,0.01,...,153,19.5,28.2,4.3,NaN,141.53,176.0,3955,3322.0,22.50
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.3,...,345,92.7,NaN,NaN,NaN,NaN,282.0,46,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yemen,937,NaN,0.1,6.8,NaN,0.455,1.95,NaN,98,0.01,...,53,13.7,23.0,5.2,1294.0,43.97,35.0,NaN,2154.0,23.85
Zambia,985,NaN,4.8,5.4,NaN,0.565,4.86,NaN,95,95.5,...,123,52.3,18.7,1.3,NaN,9.71,44.0,NaN,2002.0,21.40
Zanzibar,1031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zimbabwe,1466,NaN,4.8,7.5,NaN,0.593,3.16,NaN,88,87.0,...,127,81.1,19.9,6.1,848.0,31.90,71.0,232,1908.0,21.00


In [216]:
# b.
missing_entries_count = joined_df.isna().sum().sum()
countries_count = len(joined_df.index)


for i,col in enumerate(joined_df.columns):
    num_df = pd.to_numeric(joined_df[col], errors='coerce')
    col_data = {}
    col_data['mean'] = num_df.mean()
    col_data['median'] = num_df.median()
    col_data['max'] = num_df.max()
    col_data['mode'] = num_df.mode()
    col_data['min'] = num_df.min()
    col_data['variance'] = num_df.var()
    col_data['missingEntries'] = joined_df[col].isna().sum()
    metadata[col] = col_data


# c
joined_df.dropna(thresh=26, inplace=True) # should be 29 but removed 1 col
# joined_df.columns
joined_df = joined_df.apply(pd.to_numeric)

In [217]:
joined_df

,"(United Nations, Estimate)",Averagedownloadspeed(Mbit/s)(Ookla),2016,Rate,% of GDP,"(HDI, 2021 data (2022 report)​)",2020,"(Age 25–34, Equivalent to a 2-year degree or higher (%))","Yes, important",% Christian,...,"Rate per 100,000","(Elderly(65+), Rate)",Women,Expenditure on education(% of GDP),Homelessper 10k,Milk consumption 2013 (kg/capita/yr),Scientific publications per capita (in ppm),Titles,"(Average daily dietary energy consumption per capita, kilocalories)",Average yearly temperature (1961–1990 Celsius)
Afghanistan,508,NaN,0.2,6.7,NaN,0.478,2.85,NaN,97.0,0.02,...,87.0,20.3,15.5,4.1,1180.0,62.23,9.0,2795.0,2040.0,12.60
Albania,5181,45.25,7.5,2.1,NaN,0.796,6.08,NaN,50.0,17.00,...,164.0,86.9,24.0,4.0,NaN,303.72,216.0,390.0,3360.0,11.40
Algeria,3368,NaN,0.9,1.3,NaN,0.745,3.77,NaN,95.0,0.01,...,153.0,19.5,28.2,4.3,NaN,141.53,176.0,3955.0,3322.0,22.50
Angola,1896,NaN,6.4,4.8,NaN,0.586,3.66,NaN,88.0,75.00,...,93.0,27.0,21.4,3.5,NaN,12.30,5.0,22.0,2385.0,21.55
Argentina,8476,51.51,9.8,5.3,NaN,0.842,6.95,NaN,65.0,77.00,...,230.0,97.9,33.2,5.5,NaN,195.08,341.0,28010.0,3307.0,14.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,3740,NaN,5.6,36.7,NaN,0.691,2.76,NaN,79.0,88.00,...,134.0,88.1,22.7,3.6,NaN,117.79,49.0,2275.0,2120.0,25.35
Vietnam,2786,66.38,8.3,1.5,NaN,0.703,2.94,NaN,15.0,8.20,...,128.0,76.9,22.8,4.3,16.6,16.36,177.0,24589.0,3025.0,24.45
Yemen,937,NaN,0.1,6.8,NaN,0.455,1.95,NaN,98.0,0.01,...,53.0,13.7,23.0,5.2,1294.0,43.97,35.0,NaN,2154.0,23.85
Zambia,985,NaN,4.8,5.4,NaN,0.565,4.86,NaN,95.0,95.50,...,123.0,52.3,18.7,1.3,NaN,9.71,44.0,NaN,2002.0,21.40


In [218]:
# d

for col in joined_df.columns:
    median = metadata[col]['median']
    joined_df[col] = joined_df[col].fillna(float(median))


In [219]:
joined_df

,"(United Nations, Estimate)",Averagedownloadspeed(Mbit/s)(Ookla),2016,Rate,% of GDP,"(HDI, 2021 data (2022 report)​)",2020,"(Age 25–34, Equivalent to a 2-year degree or higher (%))","Yes, important",% Christian,...,"Rate per 100,000","(Elderly(65+), Rate)",Women,Expenditure on education(% of GDP),Homelessper 10k,Milk consumption 2013 (kg/capita/yr),Scientific publications per capita (in ppm),Titles,"(Average daily dietary energy consumption per capita, kilocalories)",Average yearly temperature (1961–1990 Celsius)
Afghanistan,508,87.01,0.2,6.7,2.1,0.478,2.85,41.0,97.0,0.02,...,87.0,20.3,15.5,4.1,1180.0,62.23,9.0,2795.0,2040.0,12.60
Albania,5181,45.25,7.5,2.1,2.1,0.796,6.08,41.0,50.0,17.00,...,164.0,86.9,24.0,4.0,34.0,303.72,216.0,390.0,3360.0,11.40
Algeria,3368,87.01,0.9,1.3,2.1,0.745,3.77,41.0,95.0,0.01,...,153.0,19.5,28.2,4.3,34.0,141.53,176.0,3955.0,3322.0,22.50
Angola,1896,87.01,6.4,4.8,2.1,0.586,3.66,41.0,88.0,75.00,...,93.0,27.0,21.4,3.5,34.0,12.30,5.0,22.0,2385.0,21.55
Argentina,8476,51.51,9.8,5.3,2.1,0.842,6.95,41.0,65.0,77.00,...,230.0,97.9,33.2,5.5,34.0,195.08,341.0,28010.0,3307.0,14.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,3740,87.01,5.6,36.7,2.1,0.691,2.76,41.0,79.0,88.00,...,134.0,88.1,22.7,3.6,34.0,117.79,49.0,2275.0,2120.0,25.35
Vietnam,2786,66.38,8.3,1.5,2.1,0.703,2.94,41.0,15.0,8.20,...,128.0,76.9,22.8,4.3,16.6,16.36,177.0,24589.0,3025.0,24.45
Yemen,937,87.01,0.1,6.8,2.1,0.455,1.95,41.0,98.0,0.01,...,53.0,13.7,23.0,5.2,1294.0,43.97,35.0,2216.5,2154.0,23.85
Zambia,985,87.01,4.8,5.4,2.1,0.565,4.86,41.0,95.0,95.50,...,123.0,52.3,18.7,1.3,34.0,9.71,44.0,2216.5,2002.0,21.40


In [220]:
# number of NaN values
joined_df.isnull().sum().sum()

0

In [221]:
# e. binary version

binary_df = joined_df.copy()

for col in binary_df.columns:
    median = metadata[col]['median']
    binary_df[col] = np.where(binary_df[col] <= median, 0, 1)
    
binary_df

,"(United Nations, Estimate)",Averagedownloadspeed(Mbit/s)(Ookla),2016,Rate,% of GDP,"(HDI, 2021 data (2022 report)​)",2020,"(Age 25–34, Equivalent to a 2-year degree or higher (%))","Yes, important",% Christian,...,"Rate per 100,000","(Elderly(65+), Rate)",Women,Expenditure on education(% of GDP),Homelessper 10k,Milk consumption 2013 (kg/capita/yr),Scientific publications per capita (in ppm),Titles,"(Average daily dietary energy consumption per capita, kilocalories)",Average yearly temperature (1961–1990 Celsius)
Afghanistan,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
Albania,0,0,1,0,0,1,1,0,0,0,...,1,1,0,0,0,1,0,0,1,0
Algeria,0,0,0,0,0,1,0,0,1,0,...,1,0,1,1,0,1,0,1,1,1
Angola,0,0,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Argentina,1,0,1,1,0,1,1,0,0,1,...,1,1,1,1,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,1,0,1,0,1
Vietnam,0,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,1,1
Yemen,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,1
Zambia,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [222]:
# PCA
df = joined_df.copy()
X = df.iloc[:,:].values
Y = df.index.values
d = {'Country': Y}
country_df = pd.DataFrame(data=d)

In [223]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pcs = pca.fit_transform(X)
pc_df = pd.DataFrame(data=pcs, columns=['pc1','pc2'])
pc_df = pd.concat([pc_df, country_df],axis=1)
pc_df

,pc1,pc2,Country
0,-21103.358169,-14874.178802,Afghanistan
1,-18312.442279,-16106.047806,Albania
2,703.241027,-12637.991886,Algeria
3,9850.341450,-16649.035413,Angola
4,-14034.656065,11977.313631,Argentina
...,...,...,...
158,-3152.746797,-14445.915641,Venezuela
159,-23081.357965,6942.240703,Vietnam
160,-22207.291609,-15255.887024,Yemen
161,-21057.345679,-15376.846425,Zambia


In [224]:
fig, ax = plt.subplots(figsize=(200, 200))
ax.scatter(x=pc_df['pc1'],y=pc_df['pc2'],c='Black')
ax.set_xlabel('Principal component 1')
ax.set_ylabel('Principal component 2')

# annotation
for i, row in pc_df.iterrows():
    ax.annotate(row['Country'], (row['pc1'], row['pc2']))


# plt.savefig('filename.png', dpi=1200)
plt.show()

In [225]:
pca5 = PCA(n_components=5)
pcs5 = pca5.fit_transform(X)
pc_df5 = pd.DataFrame(data=pcs5, columns=['pc1','pc2','pc3','pc4','pc5'])
pc_df5 = pd.concat([pc_df5, country_df],axis=1)
pc_df5

,pc1,pc2,pc3,pc4,pc5,Country
0,-21103.358169,-14874.178802,-10376.941216,876.105316,-119.494691,Afghanistan
1,-18312.442279,-16106.047806,-4883.630406,-982.257520,1239.380909,Albania
2,703.241027,-12637.991886,-8882.052461,341.452035,203.882481,Algeria
3,9850.341450,-16649.035413,-10389.543976,867.588262,-35.661417,Angola
4,-14034.656065,11977.313631,-6394.142556,-3134.575506,-299.005080,Argentina
...,...,...,...,...,...,...
158,-3152.746797,-14445.915641,-8404.053103,2173.117728,-159.071712,Venezuela
159,-23081.357965,6942.240703,-12063.102274,1300.711715,384.238488,Vietnam
160,-22207.291609,-15255.887024,-9368.388827,-733.353751,-290.485805,Yemen
161,-21057.345679,-15376.846425,-9878.126994,1222.431323,-232.636717,Zambia


In [226]:
corr_df = joined_df.copy()
corr_df = corr_df.corr(method ='pearson')
np.fill_diagonal(corr_df.values,np.nan)

corr_df

,"(United Nations, Estimate)",Averagedownloadspeed(Mbit/s)(Ookla),2016,Rate,% of GDP,"(HDI, 2021 data (2022 report)​)",2020,"(Age 25–34, Equivalent to a 2-year degree or higher (%))","Yes, important",% Christian,...,"Rate per 100,000","(Elderly(65+), Rate)",Women,Expenditure on education(% of GDP),Homelessper 10k,Milk consumption 2013 (kg/capita/yr),Scientific publications per capita (in ppm),Titles,"(Average daily dietary energy consumption per capita, kilocalories)",Average yearly temperature (1961–1990 Celsius)
"(United Nations, Estimate)",NaN,0.562129,0.430619,-0.234044,0.065994,0.711603,0.599265,0.278434,-0.590882,0.123108,...,-0.156329,0.258917,0.629469,0.221895,-0.193087,0.632632,0.881018,0.249320,0.599547,-0.385477
Averagedownloadspeed(Mbit/s)(Ookla),0.562129,NaN,0.291825,-0.204839,0.014172,0.369275,0.321101,0.286568,-0.387118,0.025105,...,-0.107548,0.054213,0.299855,0.143855,-0.033637,0.218023,0.504879,0.305199,0.282163,-0.200946
2016,0.430619,0.291825,NaN,-0.014346,-0.128741,0.511769,0.549709,0.128729,-0.617634,0.558972,...,0.134753,0.527267,0.486508,0.211240,-0.232634,0.478837,0.485446,0.167699,0.383712,-0.498782
Rate,-0.234044,-0.204839,-0.014346,NaN,-0.051234,-0.182583,0.023631,-0.209896,0.169367,0.314172,...,0.292162,0.020505,-0.045838,0.131789,0.126383,-0.183060,-0.274221,-0.113328,-0.235552,0.248722
% of GDP,0.065994,0.014172,-0.128741,-0.051234,NaN,0.070617,-0.089179,-0.022057,0.064246,-0.127852,...,0.063761,-0.030707,-0.107440,0.055809,-0.016049,-0.022776,0.029083,0.030379,0.092170,0.081310
"(HDI, 2021 data (2022 report)​)",0.711603,0.369275,0.511769,-0.182583,0.070617,NaN,0.634954,0.104037,-0.656598,0.178982,...,0.197728,0.690873,0.773794,0.245200,-0.296290,0.712373,0.723475,0.278274,0.769951,-0.517621
2020,0.599265,0.321101,0.549709,0.023631,-0.089179,0.634954,NaN,0.065201,-0.523998,0.414859,...,0.046361,0.403452,0.639829,0.307277,-0.319494,0.557992,0.649619,0.163595,0.486947,-0.359265
"(Age 25–34, Equivalent to a 2-year degree or higher (%))",0.278434,0.286568,0.128729,-0.209896,-0.022057,0.104037,0.065201,NaN,-0.300116,-0.038197,...,-0.138930,-0.048096,0.043208,-0.048210,-0.020338,0.104560,0.210525,0.040934,0.028299,-0.192945
"Yes, important",-0.590882,-0.387118,-0.617634,0.169367,0.064246,-0.656598,-0.523998,-0.300116,NaN,-0.200568,...,0.043348,-0.504160,-0.567677,-0.338228,0.182805,-0.634748,-0.652762,-0.235331,-0.527494,0.599250
% Christian,0.123108,0.025105,0.558972,0.314172,-0.127852,0.178982,0.414859,-0.038197,-0.200568,NaN,...,0.174352,0.343522,0.203209,0.140064,-0.118622,0.190204,0.172785,-0.028308,0.022667,-0.126679


In [227]:
abs_corr_df = corr_df.copy()
corr_list = abs_corr_df.abs().idxmax()
list_correlation = list(corr_list)

In [228]:
list_correlation_indices = []
for e in list_correlation:
    list_correlation_indices.append(joined_df.columns.get_loc(e))

In [210]:
data = {'named_correlation':list_correlation_indices}
with open('C:/Users/ianpe/Desktop/list_correlation_indices.json', 'w') as f:
    json.dump(data, f)

In [229]:
# 3. predictions using bayes classifier (accuracy = exactitude = (TN + TP) / Total)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

score = []
feature_importance = []

for col in binary_df.columns:
    X = binary_df.drop(col, axis=1).values
    Y = binary_df[col].values
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=86)
    clf = BernoulliNB()
    clf.fit(X_train, y_train)
    # 2 most predictive features given class 0 and 1
#     f0 = clf.feature_log_prob_[0, :].argsort()[::-1][:2]
#     f1 = clf.feature_log_prob_[1, :].argsort()[::-1][:2]
    y_pred = clf.predict(X_test)
    score.append(accuracy_score(y_test, y_pred))
#     feature_importance.append([f0,f1])

# print(accuracy_score(y_test, y_pred))
# print(precision_score(y_test, y_pred))
# print(best_cols)

# a
score

# c
# feature_importance

[0.8787878787878788,
 0.6666666666666666,
 0.7272727272727273,
 0.7272727272727273,
 0.8181818181818182,
 0.8787878787878788,
 0.8181818181818182,
 0.7272727272727273,
 0.7878787878787878,
 0.7272727272727273,
 0.7272727272727273,
 0.9696969696969697,
 0.9696969696969697,
 0.6060606060606061,
 0.7878787878787878,
 0.6666666666666666,
 0.6363636363636364,
 0.9393939393939394,
 0.6363636363636364,
 0.8484848484848485,
 0.8181818181818182,
 0.8181818181818182,
 0.8181818181818182,
 0.5151515151515151,
 0.7878787878787878,
 0.9393939393939394,
 0.8484848484848485,
 0.5757575757575758,
 0.5757575757575758,
 0.7272727272727273,
 0.6060606060606061,
 0.7272727272727273,
 0.6666666666666666,
 0.9393939393939394,
 0.7272727272727273,
 0.8484848484848485,
 0.6666666666666666]

In [230]:
# paire de colonnes qui permet predire x avec la meilleure précision.
best_precision_pairs = []
for i_col, col in enumerate(binary_df.columns):
    best_precision_score = 0
    best_cols = []
    for i in range(len(binary_df.columns)-2):
        for j in range(i+1,len(binary_df.columns)-1):
            if i != i_col and j != i_col:
                X = binary_df.iloc[:, [i,j]].values
                Y = binary_df.iloc[:,[i_col]].values
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=86)
                clf = BernoulliNB()
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)
                score = precision_score(y_test, y_pred)
                if score > best_precision_score:
                    best_precision_score = score
                    best_cols = [i,j]
    best_precision_pairs.append(best_cols)
    
data = {'best_precision_pairs':best_precision_pairs}

In [202]:
# import json
data = {'score':score}
with open('C:/Users/ianpe/Desktop/score.json', 'w') as f:
    json.dump(data, f)

In [231]:
# c
# normalize data first with cote Z
# from sklearn.naive_bayes import GaussianNB

# # normalized_binary_df = (binary_df - binary_df.mean()) / binary_df.std()

# for col in normalized_binary_df.columns:
#     X = normalized_binary_df.drop(col, axis=1).values
#     y = normalized_binary_df[col].values
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=86)
#     clf = GaussianNB()
#     # clf = BernoulliNB()
#     clf.fit(X_train, y_train)
#     print(clf.feature_log_prob_)
#     # 2 most predictive features given class 0 and 1
#     f0 = clf.feature_log_prob_[0, :].argsort()[::-1][:2]
#     f1 = clf.feature_log_prob_[1, :].argsort()[::-1][:2]
#     print(f0, ' ',f1)
#     y_pred = clf.predict(X_test)
#     score.append(accuracy_score(y_test, y_pred))
#     feature_importance.append([f0,f1])

In [232]:
# 3. predictions using linear regression MSE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

score = []
feature_importance = []

linear_df = joined_df.copy()

for col in linear_df.columns:
    X = linear_df.drop(col, axis=1).values
    y = linear_df[col].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=86)
    clf = LinearRegression()
    clf.fit(X_train, y_train)
    # 2 most predictive features given class
    coeffs = list(clf.coef_)
    first_best_index = coeffs.index(max(coeffs))
    coeffs.pop(first_best_index)
    scd_best_index = coeffs.index(max(coeffs))
    y_pred = clf.predict(X_test)
    score.append(mean_squared_error(y_test, y_pred))
    feature_importance.append([first_best_index,scd_best_index])

# print(accuracy_score(y_test, y_pred))
# print(precision_score(y_test, y_pred))
# print(best_cols)

# a
score

# c
# feature_importance

[35560791.720990054,
 1442.7943765160637,
 4.838450166520042,
 47.471193906550226,
 0.19779718380206351,
 0.0020017410353879903,
 2.1880711060441387,
 60.343492108115434,
 327.3213690786707,
 557.2805682725502,
 458.71234130990956,
 118.12275767221703,
 112.96133455519644,
 6.776550765664936,
 33397314.586548045,
 20783.04363491346,
 33.32716819431018,
 442220.7486628676,
 36.420989017891536,
 0.27158791078922245,
 473754.9315200351,
 44.72735312572419,
 60.7477148669005,
 1466001698.2042902,
 0.34461884757931344,
 6.65442477642144,
 538.3027288981659,
 18336.123362897488,
 452.9632562727023,
 6.585216274111699,
 3.4703236954612477,
 148660.97982886015,
 4458.709052470766,
 390962.26087023894,
 2210095988.0787063,
 84931.6066140303,
 46.96104099244118]

In [200]:
data = {'score':score}
with open('C:/Users/ianpe/Desktop/score2.json', 'w') as f:
    json.dump(data, f)